# Collaborative Filtering with Neural Nets

In [2]:
# for data manipulation
import numpy as np
import pandas as pd
import os
import pickle

# use surprise for collaborative filtering
import torch
import torch.nn as nn
from torch.autograd import Variable

## Read in data

In [28]:
game_data_path = "data/neural_net_data/"
files = sorted(os.listdir(game_data_path))
start = 6
with open(game_data_path + files[start], 'rb') as f:
    print(files[start])
    X, y = pickle.load(f, encoding='latin1')
for i in range(1):
    with open(game_data_path + files[start + 1 + i], 'rb') as f:
        print(files[start + 1 + i])
        X_add, y_add = pickle.load(f, encoding='latin1')
        print(X.shape)
        print(X_add.shape)
        X = np.concatenate((X, X_add), axis = 0)
        y = np.concatenate((y, y_add), axis = 0)

2014-2015.pkl
2015-2016.pkl
(1263, 3, 504)
(1269, 3, 504)


In [31]:
X.shape

(2476, 3, 504)

In [20]:
y_val.shape

torch.Size([1241, 1])

In [5]:
def trainValSplit(X, y):
    X = np.array(X)
    y = np.array(y)

    X = X[y > 0]
    y = y[y > 0]

    p = np.random.permutation(len(X))
    X = X[p]
    y = y[p]

    val = 0.2
    val = round(len(X) * val)
    X_val = X[:val]
    y_val = y[:val]
    X = X[val:]
    y = y[val:]
    
    return X, y, X_val, y_val

In [6]:
X, y, X_val, y_val = trainValSplit(X, y)

In [29]:
# BySeason trainValSplit
game_data_path = "data/neural_net_data/"
files = sorted(os.listdir(game_data_path))
with open(game_data_path + files[7], 'rb') as f:
    print(files[7])
    X_val, y_val = pickle.load(f, encoding='latin1')
    
X = X[y > 0]
y = y[y > 0]
X_val = X_val[y_val > 0]
y_val = y_val[y_val > 0]

2015-2016.pkl


In [32]:
y = torch.from_numpy(y[:,np.newaxis]).type(torch.FloatTensor)
X = torch.from_numpy(X.reshape((X.shape[0], -1))).type(torch.FloatTensor)
y_val = torch.from_numpy(y_val[:,np.newaxis]).type(torch.FloatTensor)
X_val = torch.from_numpy(X_val.reshape((X_val.shape[0], -1))).type(torch.FloatTensor)

In [33]:
# Split train/test:
print(y[:10])

tensor([[ 208.],
        [ 197.],
        [ 188.],
        [ 168.],
        [ 218.],
        [ 191.],
        [ 181.],
        [ 188.],
        [ 189.],
        [ 185.]])


In [34]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt

class Net(torch.nn.Module):
    def __init__(self, n_feature, n_output):
        super(Net, self).__init__()
        
        # layer 1 fully connected 150 units
        self.lin1 = nn.Linear(n_feature, 500)
        
        # layer 2 fully connected 50 units
        self.lin2 = nn.Linear(500, 100)
        
        # layer 3 fully connected 1 unit (output)
        self.lin3 = nn.Linear(100, n_output)
        
        # self.lin4 = nn.Linear(50, n_output)
        
        # dropouts
        self.drop1 = nn.Dropout(0.5)
        self.drop2 = nn.Dropout(0.4)
        self.drop3 = nn.Dropout(0.25)

    def forward(self, x):
        # perform dropout on input vector embeddings
        # x = self.drop1(x)
        x = F.relu(self.lin1(x))
        # x = self.drop2(F.relu(self.lin1(x)))
        x = F.relu(self.lin2(x))
        # x = self.drop3(F.relu(self.lin2(x)))
        x = self.lin3(x)
        
        return x  

net = Net(n_feature=1512, n_output=1)     # define the network
print(net)  # net architecture

optimizer = torch.optim.SGD(net.parameters(), lr=0.000001)
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss

# plt.ion()   # something about plotting

for t in range(10000):
    prediction = net(X)     # input x and predict based on x

    loss = loss_func(prediction, y)     # must be (1. nn output, 2. target)

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients
    
    # Do validation loss
    with torch.no_grad():
        pred_val = net(X_val)
        loss_val = loss_func(pred_val, y_val)

    if t % 100 == 0:
        # plot and show learning process
        '''
        plt.cla()
        plt.scatter(x.data.numpy(), y.data.numpy())
        plt.plot(x.data.numpy(), prediction.data.numpy(), 'r-', lw=5)
        plt.text(0.5, 0, 'Loss=%.4f' % loss.data.numpy(), fontdict={'size': 20, 'color':  'red'})
        plt.pause(0.1)
        '''
        print("-----")
        print(t)
        print('Loss=%.4f' % loss.data.numpy())
        print('Val Loss=%.4f' % loss_val.data.numpy())

# plt.ioff()
# plt.show()

Net(
  (lin1): Linear(in_features=1512, out_features=500, bias=True)
  (lin2): Linear(in_features=500, out_features=100, bias=True)
  (lin3): Linear(in_features=100, out_features=1, bias=True)
  (drop1): Dropout(p=0.5)
  (drop2): Dropout(p=0.4)
  (drop3): Dropout(p=0.25)
)
-----
0
Loss=42639.1914
Val Loss=9045.6992
-----
100
Loss=889.0920
Val Loss=1135.7629
-----
200
Loss=670.3543
Val Loss=793.4302
-----
300
Loss=593.2773
Val Loss=677.2971
-----
400
Loss=538.7346
Val Loss=596.7897
-----
500
Loss=507.6362
Val Loss=551.5566
-----
600
Loss=482.6397
Val Loss=516.6025
-----
700
Loss=464.2978
Val Loss=491.3290
-----
800
Loss=448.8751
Val Loss=470.2880
-----
900
Loss=433.8632
Val Loss=450.2365
-----
1000
Loss=422.1149
Val Loss=435.1807
-----
1100
Loss=414.2003
Val Loss=424.8576
-----
1200
Loss=404.8118
Val Loss=412.9027
-----
1300
Loss=402.4381
Val Loss=409.9925
-----
1400
Loss=392.1365
Val Loss=397.4779
-----
1500
Loss=387.4836
Val Loss=391.8299
-----
1600
Loss=382.7631
Val Loss=386.1480
---